# Workshop MQTT - IoT
## Taller práctico de MQTT aplicado a IoT

### Objetivo del Taller
Ofrecer a los participantes una comprensión fundamental de los protocolos de comunicación en el Internet de las Cosas (IoT), con un enfoque práctico en MQTT. Al finalizar, los asistentes podrán entender la arquitectura de MQTT, sus componentes principales y cómo implementar una comunicación básica entre dispositivos utilizando Python.

## 1. Preparación del Taller 

Llegado a este punto, ya deberías haber ejecutado los comandos de los archivos `README.md` y `Containers.md`, con lo cual deberías tener 3 contenedores corriendo en tu máquina local.

Puedes verificar que los contenedores están corriendo ejecutando el siguiente commando en la terminal de tu computadora:

<hr> 

```bash
docker ps
```

<hr>

Deberías ver algo similar a esto:
<hr>

```text
CONTAINER ID   IMAGE                           ...   PORTS                                         NAMES
323c7c94d561   jupyter/scipy-notebook:latest   ...   0.0.0.0:8888->8888/tcp, [::]:8888->8888/tcp   jlab
2a56c3f7f377   nodered/node-red:latest         ...   0.0.0.0:1880->1880/tcp, [::]:1880->1880/tcp   nodered
c1322f110427   eclipse-mosquitto               ...   0.0.0.0:1883->1883/tcp, [::]:1883->1883/tcp   mosquitto
```

<hr>
Para este laboratorio, estaremos utilizando el broker MQTT de Eclipse Mosquitto, pero puedes elegir cualquier otro broker de tu preferencia, por ejemplo:

- [Eclipse Mosquitto](https://mosquitto.org/): Un broker MQTT de código abierto y ligero.
- [HiveMQ](https://www.hivemq.com/): Un broker MQTT comercial que ofrece una versión comunitaria.
- [EMQX](https://www.emqx.io/): Un broker MQTT escalable y de alto rendimiento.
- [NanoMQ](https://nanomq.io/): Un broker MQTT ultraligero diseñado para dispositivos IoT.

## 2. Preparación del Entorno Python

Para este taller ocupamos algunas dependencias de Python, para trabajar con el protocolo MQTT. Para instalarlas todas las dependencias dejaremos que el comando `pip` lea el archivo e instale todo lo necesario.

Ejecuta el siguiente bloque.

In [ ]:
%pip install paho-mqtt

## 2. Manos a la obra

### 2.1. Variables de Entorno

Definiremos las siguientes variables que usaremos en los siguientes códigos, debes ejecutar el siguiente bloque de código para definirlas en el kernel de Jupyter.

In [ ]:
BROKER_ADDRESS = "mosquitto"  # Dirección del broker MQTT
USERNAME = "nodered"          # Usuario para autenticación
PASSWORD = "nodered"          # Contraseña para autenticación
PORT = 1883                   # Puerto del broker MQTT
QOS_LEVEL = 2                 # Quality of Service, 0 = At most once, 1 = At least once, 2 = Exactly once
TOPIC = "iot/workshop/sensor/sala1" # Tópico base

### 2.2. Importación de Librerías

Importaremos en el siguiente bloque las librerías requeridas de Python que se requieren para los siguientes bloques de código, por favor ejecuta lo siguiente para cargarlas en el kernel de Jupyter

In [ ]:
# Librería para conexión MQTT
import paho.mqtt.client as mqtt
from paho.mqtt.enums import CallbackAPIVersion

# Librería para gestión de Tiempo
import time

# Librería para manejo de cadenas JSON
import json

# Añadir path para modulos del workshop
import sys
sys.path.append('/home/jovyan/work')

### 2.3. Enviar un mensaje a MQTT

El siguiente bloque de código muestra un ejemplo para enviar un mensaje al broker MQTT en el topic `iot/workshop/sensor/sala1/temperatura`. El bloque enviará un único mensaje, pero puedes ejecutar el bloque las veces que desees. Recuerda mantener el MQTT Explorer abierto para observar los mensajes que se publican.

En el MQTT Explorer, te puedes suscribir al topic `iot/workshop/sensor/sala1/#` para ver todos los mensajes publicados en los subtopics de `sala1`, el caracter `#` es un comodín que indica "todos los subtopics".

In [ ]:
# Inicialización del cliente MQTT, en versión MQTT 5
client = mqtt.Client(
    protocol=mqtt.MQTTv5,
    callback_api_version=CallbackAPIVersion.VERSION2
)

# Establecimiento de credenciales del broker
client.username_pw_set(username=USERNAME, password=PASSWORD)

In [ ]:
# Conexión al broker MQTT
client.connect(
    host=BROKER_ADDRESS, 
    port=PORT,
    keepalive=60
)

# Iniciar el loop para procesar eventos
client.loop_start()  

# Definición del mensaje, en este caso usamos un mensaje simple de texto, puedes variar este texto
mensaje = f"Temperatura: 23.5°C\nTS: {time.time()}"

# Publicación del mensaje en el topic especificado con el nivel de QoS definido
client.publish(
    topic=f"{TOPIC}/temperatura", 
    payload=mensaje, 
    qos=QOS_LEVEL
)

# Desconectar el cliente MQTT
client.disconnect()

# Detener el loop
client.loop_stop()

Debido a que en IoT es común que el contenido del mensaje sea utilizado para análisis posteriores, es recomendable utilizar un formato estructurado como JSON para los mensajes. Esto facilita la interpretación y el procesamiento de los datos en sistemas posteriores, por ejemplo, podemos cambiar el ejemplo anterior a lo siguiente:

In [ ]:
# Conexión al broker MQTT
client.connect(
    host=BROKER_ADDRESS, 
    port=PORT,
    keepalive=60
)

# Iniciar el loop para procesar eventos
client.loop_start()  

# Definición del mensaje, en este caso lo definimos como un diccionario y luego lo convertimos a JSON
mensaje = {
    "value": 23.5,
    "unit": "°C",
    "timestamp": time.time()
}

# Publicación del mensaje en el topic especificado con el nivel de QoS definido
client.publish(
    topic=f"{TOPIC}/temperatura", 
    payload=json.dumps(mensaje), 
    qos=QOS_LEVEL
)

# Desconectar el cliente MQTT
client.disconnect()

# Detener el loop
client.loop_stop()

### 2.2. Simulación de un sensor

En el siguiente ejercicio, vamos a simular un sensor que envía datos de temperatura y humedad cada 3 segundos. 
En la carpeta de este projecto, hay un archivo llamado `sensor.py` que contiene el código necesario para esta simulación. Puedes abrirlo y revisarlo para entender cómo funciona.

Para visualizar los mensajes que envía este sensor, asegúrate de tener el MQTT Explorer abierto y suscrito al topic `iot/workshop/sensor/sala1/#`, y ejecuta el siguiente bloque de código.


In [ ]:
measures = [
    # Nombre medida, unidad, valor mínimo, valor máximo
    ("temperatura", "°C", 20, 25), 
    ("humidity", "%", 30, 50) 
]

In [ ]:
from sensor import generar_datos_sensor

# Conexión al broker MQTT
client.connect(
    host=BROKER_ADDRESS, 
    port=PORT,
    keepalive=60
)

# Iniciar el loop para procesar eventos
client.loop_start()  

while True:
    try:
        for nombre, unidad, vmin, vmax in measures:
            valor = generar_datos_sensor(nombre, vmin, vmax)
            
            mensaje = {
                "value": valor,
                "unidad": unidad,
                "timestamp": time.time()
            }
            
            # Publicación del mensaje en el topic especificado con el nivel de QoS definido
            client.publish(
                topic=f"{TOPIC}/{nombre}", 
                payload=json.dumps(mensaje), 
                qos=QOS_LEVEL
            )
            
        time.sleep(3)  # Esperar 3 segundos antes de enviar la siguiente medida
    except KeyboardInterrupt:
        break

# Desconectar el cliente MQTT
client.disconnect()
client.loop_stop()

### 2.3. Prueba con más valores

En el archivo `medidas_pruebas.csv` tenemos una colección de medidas que podríamos simular en un entorno IoT. Cada fila del archivo contiene el nombre de la medida, la unidad, el valor mínimo y el valor máximo.

Puedes abrir el archivo `medidas_pruebas.csv` y revisar su contenido. Luego, puedes modificar el bloque de código anterior para que envíe datos para cada una de estas medidas, en lugar de solo temperatura y humedad.

Si ocupas cargar el archivo CSV, puedes utilizar el siguiente bloque de código como referencia:

In [ ]:
import csv

with open("/home/jovyan/work/medidas_pruebas.csv", mode='r') as file:
    reader = csv.reader(file)
    measures = list(reader)

measures = [(name, unit, float(vmin), float(vmax)) for name, unit, vmin, vmax in measures[1:]] # Omitir la cabecera

print(f"Cargadas {len(measures)} medidas de prueba")

# 3. Node-RED

Node-RED es una herramienta de desarrollo basada en flujo, ideal para conectar dispositivos de hardware, APIs y servicios en línea de nuevas maneras. Proporciona un entorno visual para programar mediante la conexión de bloques funcionales llamados "nodos".

En la siguiente etapa de este taller, vamos a utilizar Node-RED para suscribirnos a los mensajes que estamos enviando al broker MQTT y procesarlos de alguna manera.

Tienes acceso a la presentación de Node-RED por medio del siguiente enlace: 

[https://docs.google.com/presentation/d/10JEMlOmMqKfLAdnGXZqYSK0AzVqW7HDctlsPz-MMQuQ/edit?usp=sharing](https://docs.google.com/presentation/d/10JEMlOmMqKfLAdnGXZqYSK0AzVqW7HDctlsPz-MMQuQ/edit?usp=sharing)

Para la configuración de Node-RED, toma como referencia las siguiente imágenes:

## 3.1. Configuración del nodo MQTT

![Configuración del nodo MQTT](../images/mqtt_config.jpg)

## 3.2. Instalación del complemento para Email

![Instalación Email](../images/email_installation.jpg)

## 3.3. Configuración del nodo Email

![Configuración Email](../images/email_config.jpg)